In [8]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict
from langchain.chat_models import init_chat_model
from langgraph.graph.message import MessagesState


llm = init_chat_model("openai:gpt-4o-mini")
llm.invoke([{"role": "user", "content": "Hello, how are you?"}])

AIMessage(content="Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 13, 'total_tokens': 42, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_373a14eb6f', 'id': 'chatcmpl-DB50njsXLuKEZQJPQCAZKv26flkFC', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--019c779c-185a-74e0-b828-47f7cd3e45ec-0', usage_metadata={'input_tokens': 13, 'output_tokens': 29, 'total_tokens': 42, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [9]:
class State(MessagesState):
    custom_stuff: str


graph_builder = StateGraph(State)

In [10]:
def chatbot(state: State):
    response = llm.invoke(state["messages"])
    return {
        "messages": [response]
    }


graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)


In [11]:
graph = graph_builder.compile()

graph.invoke(
    {
        "messages": [
        {"role": "user", "content": "Hello, how are you?"}
        ]
    }
)

{'messages': [HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={}, id='be699f69-b5c0-454f-82da-9ba3f0eb947e'),
  AIMessage(content="Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 13, 'total_tokens': 42, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_373a14eb6f', 'id': 'chatcmpl-DB50oZwn7SISXS7hA8d7GkqLdMzn7', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--019c779c-2108-73a0-9d7c-7eba9d9c8876-0', usage_metadata={'input_tokens': 13, 'output_tokens': 29, 'total_tokens': 42, 'input_token_details': {'audio': 0, 'cache_re